# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [1]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""   
# sql 문 마지막에 `;` 은 붙이지 않는다.

In [2]:
import pymysql
try:
    conn = None
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user="scott",        # username: str
        password="tiger",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer")
    cursor.execute(create_sql) 
    
finally:
    # 4. 연결닫기(끊기)
    if conn:                                    # 'conn = !''  이 None 이 아니라면..
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [ ]:
from datetime import date, datetime, time
a = date(2000, 10, 20)                                    # 날짜 객체 생성 - 년, 월, 일
print(a.year, a.month, a.day)                             # data객체에서 년/월/일 조회
b = datetime(2000,  10, 20, 12, 23)                       # 날짜시간 객체 생성 - 년, 월, 일, 시, 분, 초 지정. (초는 생략가능)
print(b.year, b.month, b.day, b.hour, b.minute, b.second) # 년/월/일/시/분/초 조회

print(datetime.now())  # 실행시점의 일시로 datetime객체 생성
print(date.today())    # 실행시점의 날짜로 date객체 생성

## DML

### insert


In [3]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [4]:
# connection, cursor 생성 -> with 문으로 작성.
# manual commit 이 default. 
##  DML(insert/update/delete) 처리 후 commit을 실행해야한다.
with pymysql.connect(host="127.0.0.1", port=3306, user="scott", password="tiger", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)  # 처리행수(insert/delete/update 된 행수, select 조회행수)
        print("처리 행수:", result)
        # commit 처리
        conn.commit()

처리 행수: 1


### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [6]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")

# SQL 실행
insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"
                                                                           # [Parameterized Query----------]

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, 175.23, "2010-01-01"])
        conn.commit()
        print("처리 행수:", result)

이름: 안중군
이메일주소: 12353@d.com


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [9]:
datetime.now()

datetime.datetime(2024, 10, 10, 11, 21, 7, 446060)

In [11]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [12]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)  # executemany() 결과 행수 반환.
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [13]:
update_sql = "update customer set tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  168
변경할 고객 ID:  8


처리 행수:  1


In [14]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
email = input("변경할 Email주소: ")
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 Email주소:  ytn@yy-tn.co.kr
변경할 키:  188
변경할 고객 ID:  1


처리 행수:  1


In [15]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
#       result = cursor.execute(delete_sql, (tall,))
        print("처리 행수: ", result)
        conn.commit()

삭제기준 키: 180


처리 행수:  3


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [16]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [17]:
resultset
# tuple 개별행 - tuple(조회된 컬럼값들)

((2, '유관순', 175.23),
 (3, '안중군', 175.23),
 (4, '김인영', 165.0),
 (5, '오수철', 175.0),
 (7, '김명수', 177.0),
 (8, '이지영', 168.0))

In [18]:
# 첫번째 사람의 이름
resultset[0][1]  # 인덱싱을 두번 해야함. 

'유관순'

In [19]:
resultset
# list[dictionary{key:컬럼명, value:컬럼값}]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (103692968.py, line 2)

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [20]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 6


In [21]:
resultset

[{'id': 2, 'name': '유관순', 'tall': 175.23},
 {'id': 3, 'name': '안중군', 'tall': 175.23},
 {'id': 4, 'name': '김인영', 'tall': 165.0},
 {'id': 5, 'name': '오수철', 'tall': 175.0},
 {'id': 7, 'name': '김명수', 'tall': 177.0},
 {'id': 8, 'name': '이지영', 'tall': 168.0}]

### fetchone()


In [34]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [7])  # tuple type
        print("조회행수:", result)
        resultset = cursor.fetchone()

조회행수: 1


In [35]:
resultset

(7,
 '김명수',
 'jkl@abc.com',
 177.0,
 datetime.date(2000, 2, 12),
 datetime.datetime(2024, 10, 10, 11, 23, 51))

In [37]:
if resultset:  # 조회결과가 없으면 None
    print(resultset)
   # print("이름:", resultset[0])
else:
    print("조회결과가 없음.")

(7, '김명수', 'jkl@abc.com', 177.0, datetime.date(2000, 2, 12), datetime.datetime(2024, 10, 10, 11, 23, 51))
이름: 명


### fetchmany()


In [39]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개 (2행만 조회 출력)
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)  # 조회할 자료가 없으면  빈 tuple로 출력됨

조회행수: 6


In [40]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)

((2, '유관순', datetime.date(2010, 1, 1)), (3, '안중군', datetime.date(2010, 1, 1)))
((4, '김인영', datetime.date(2005, 1, 12)), (5, '오수철', datetime.date(1995, 12, 20)))
((7, '김명수', datetime.date(2000, 2, 12)), (8, '이지영', datetime.date(1995, 4, 21)))
()


### select 문을 실행한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [44]:
ql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        for id, name, email, tall, birthday, created_at in cursor:
             print(id, name, email, tall, birthday, created_at, sep=" , ")
        # for v in cursor:
           # print(v)

조회행수: 6
2 , 유관순 , 123@adf.com , 175.23 , 2010-01-01 , 2024-10-10 11:17:45
3 , 안중군 , 12353@d.com , 175.23 , 2010-01-01 , 2024-10-10 11:18:09
4 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2024-10-10 11:23:51
5 , 오수철 , def2@a.com , 175.0 , 1995-12-20 , 2024-10-10 11:23:51
7 , 김명수 , jkl@abc.com , 177.0 , 2000-02-12 , 2024-10-10 11:23:51
8 , 이지영 , mno@abc.com , 168.0 , 1995-04-21 , 2024-10-10 11:23:51


In [41]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        # for id, name, email, tall, birthday, created_at in cursor:
          #  print(id, name, email, tall, birthday, created_at, sep=" , ")
        for v in cursor:
            print(v)

조회행수: 6
(2, '유관순', '123@adf.com', 175.23, datetime.date(2010, 1, 1), datetime.datetime(2024, 10, 10, 11, 17, 45))
(3, '안중군', '12353@d.com', 175.23, datetime.date(2010, 1, 1), datetime.datetime(2024, 10, 10, 11, 18, 9))
(4, '김인영', 'abc2@a.com', 165.0, datetime.date(2005, 1, 12), datetime.datetime(2024, 10, 10, 11, 23, 51))
(5, '오수철', 'def2@a.com', 175.0, datetime.date(1995, 12, 20), datetime.datetime(2024, 10, 10, 11, 23, 51))
(7, '김명수', 'jkl@abc.com', 177.0, datetime.date(2000, 2, 12), datetime.datetime(2024, 10, 10, 11, 23, 51))
(8, '이지영', 'mno@abc.com', 168.0, datetime.date(1995, 4, 21), datetime.datetime(2024, 10, 10, 11, 23, 51))


In [63]:
# database.py  모듈을 이용해서 hr_join DB 와 관련된 select 실행.
from database import HRDao  # Dao : Data Access Object
import configparser as parser  # config.ini 의 정보를 조회.

In [67]:
#configparser 에서 MYSQL연결 정보 조회
props = parser.ConfigParser()  # Parser 객체 생성
props.read("./config.ini")     # ini 파일 읽기
mysql_config = props['MYSQL']  # MYSQL Section "[MYSQL]"의 설정정보 조회
print("host:", mysql_config['host'])

host: 127.0.0.1


In [68]:
dao = HRDao(mysql_config['host'], 
            3306, 
            mysql_config['user'], 
            mysql_config['password'], 
            mysql_config['db'])

## 전 직원 정보를 조회 - select_all_emp()
all_emp = dao.select_all_emp()
len(all_emp)

0

In [69]:
all_dept = dao.select_dept()
print(all_dept)

((110, 'Accounting', 'Seattle'), (160, 'Benefits', 'Seattle'), (180, 'Construction', 'Seattle'), (190, 'Contracting', 'Seattle'), (140, 'Control And Credit', 'Seattle'), (130, 'Corporate Tax', 'Seattle'), (90, 'Executive', 'Seattle'), (100, 'Finance', 'Seattle'), (240, 'Government Sales', 'Seattle'), (40, 'Human Resources', 'New York'), (60, 'IT', 'San Francisco'), (230, 'IT Helpdesk', 'Seattle'), (210, 'IT Support', 'Seattle'), (170, 'Manufacturing', 'Seattle'), (20, 'Marketing', 'New York'), (220, 'NOC', 'Seattle'), (270, 'Payroll', 'Seattle'), (70, 'Public Relations', 'New York'), (30, 'Purchasing', 'Seattle'), (260, 'Recruiting', 'Seattle'), (250, 'Retail Sales', 'Seattle'), (80, 'Sales', 'New York'), (150, 'Shareholder Services', 'Seattle'), (50, 'Shipping', 'San Francisco'), (120, 'Treasury', 'Seattle'))


In [70]:
result = dao.select_emp_by_dept(10)
result

()